In [1]:
import pandas as pd

# Text Mining on persian news
    i want to learn how can i change accuracy
    of predicted class of social class ['Category']

In [2]:
data = pd.read_csv('../data/per.csv')
data.head()

,NewsID,Title,Body,Date,Time,Category,Category2
0,843656,\nوزير علوم درجمع استادان نمونه: سن بازنشستگي ...,\nوزير علوم در جمع استادان نمونه كشور گفت: از ...,\n138/5//09,\n0:9::18,\nآموزشي-,\nآموزشي
1,837144,\nگردهمايي دانش‌آموختگان موسسه آموزش عالي سوره...,\nبه گزارش سرويس صنفي آموزشي خبرگزاري دانشجويا...,\n138/5//09,\n1:4::11,\nآموزشي-,\nآموزشي
2,436862,\nنتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور...,\nنتايج آزمون دوره‌هاي فراگير مقاطع كارشناسي و...,\n138/3//07,\n1:0::03,\nآموزشي-,\nآموزشي
3,227781,\nهمايش يكروزه آسيب شناسي مفهوم روابط عمومي در...,\n,\n138/2//02,\n1:3::42,\nاجتماعي-خانواده-,\nاجتماعي
4,174187,\nوضعيت اقتصادي و ميزان تحصيلات والدين از مهمت...,\nمحمدتقي علوي يزدي، مجري اين طرح پژوهشي در اي...,\n138/1//08,\n1:1::49,\nآموزشي-,\nآموزشي


In [28]:
with open('../data/stopwords.txt') as stopwords_file:
    stopwords = stopwords_file.readlines()
stopwords = [line.replace('\n', '') for line in stopwords]
' '.join(stopwords[:50])

'! " # ( ) * , - . / : [ ] « » ، ؛ ؟ آباد آخ آخر آخرها آخه آدمهاست آرام آرام آرام آره آری آزادانه آسان آسیب پذیرند آشنایند آشکارا آقا آقای آقایان آمد آمدن آمده آمرانه آن آن گاه آنان آنانی آنجا آنرا آنطور آنقدر آنها آنهاست'

# nltk
    use nltk => natraul language tool kit
        use => stopwords and punkt of english language

In [4]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mr.rezoo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mr.rezoo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
nltk_stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(nltk_stopwords)
len(stopwords)

1495

# hazm
    use hazm => persioan natral language prosessors pakage
        use => Stemmer

In [6]:
import hazm

stemmer = hazm.Stemmer()
stemmer.stem('کتاب ها')

'کتاب '

In [7]:
lem = hazm.Lemmatizer()
lem.lemmatize('می‌روم')

'رفت#رو'

### Preprocess of news
     1 => join ['Title','Body']
     2 => tokenization vocab
     3 => filter from stopwords
     4 => stemming vocabs

In [8]:
from hazm import word_tokenize

"""
    Preprocess of news
     1 => join ['Title','Body']
     2 => tokenization vocab
     3 => filter from stopwords
     4 => stemming vocabs
"""
dataset = pd.DataFrame(columns=('title_body', 'category'))
for index, row in data.iterrows():
    title_body = row['Title'] + ' ' + row['Body']
    title_body_tokenized = word_tokenize(title_body)
    title_body_tokenized_filtered = [w for w in title_body_tokenized if not w in stopwords]
    title_body_tokenized_filtered_stemmed = [stemmer.stem(w) for w in title_body_tokenized_filtered]
    title_body_tokenized_filtered_lem = [lem.lemmatize(w).replace('#', ' ') for w in title_body_tokenized_filtered]
    dataset.loc[index] = {
        'title_body': ' '.join(title_body_tokenized_filtered_lem) + ' ' + ' '.join(
            title_body_tokenized_filtered_stemmed),
        'category': row['Category2'].replace('\n', ''),
    }

In [9]:
dataset.head()


,title_body,category
0,وزير علوم درجمع استاد نمونه سن بازنشستگي استاد...,آموزشي
1,گردهمايي دانش‌آموختگان موسسه آموزش عالي سوره ب...,آموزشي
2,نتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور ن...,آموزشي
3,همايش يكروزه آسيب شناسي مفهوم روابط عمومي بابل...,اجتماعي
4,وضعيت اقتصادي ميزان تحصيلات والدين مهمترين عوا...,آموزشي


 ### use bag of words to vectorized static title

In [10]:
"""
    use bag of words to vectorized static title

        ==>> use ngram_range
"""
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vectorizer.fit(dataset['title_body'])

TfidfVectorizer(ngram_range=(1, 2))

In [11]:
X = vectorizer.transform(dataset['title_body'])

In [12]:
X

<10999x1861001 sparse matrix of type '<class 'numpy.float64'>'
	with 6234277 stored elements in Compressed Sparse Row format>

### encoding labels | change categorical to numeric

In [13]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(dataset['category'])

In [14]:
len(y)

10999

In [15]:
label_encoder.classes_

array(['آموزشي', 'اجتماعي', 'اقتصادي', 'بهداشتي', 'تاريخي', 'سياسي',
       'علمي', 'فرهنگي', 'فقه و حقوق', 'مذهبي', 'ورزشي'], dtype=object)

In [16]:
import numpy as np

print(np.unique(dataset['category']))

['آموزشي' 'اجتماعي' 'اقتصادي' 'بهداشتي' 'تاريخي' 'سياسي' 'علمي' 'فرهنگي'
 'فقه و حقوق' 'مذهبي' 'ورزشي']


In [17]:
X.shape

(10999, 1861001)

In [18]:
np.shape(y)

(10999,)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [20]:
from sklearn import svm

svm_c = svm.SVC()
svm_c.fit(X_train, y_train)

SVC()

    check %25 data of X => X_test to predict and compare with y_test
    :return accuracy

In [21]:
"""
    check %25 data of X => X_test to predict and compare with y_test
    :return accuracy
"""
svm_c.score(X_test, y_test)


0.8483636363636363

In [22]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = svm_c.predict(X_test)
print(classification_report(y_test, y_pred))
"""
    low accuracy of class 1 => social news
"""

              precision    recall  f1-score   support

           0       0.88      0.88      0.88       241
           1       0.59      0.71      0.64       259
           2       0.80      0.84      0.82       241
           3       0.85      0.89      0.87       255
           4       0.93      0.90      0.92       238
           5       0.84      0.73      0.78       260
           6       0.79      0.73      0.76       223
           7       0.83      0.90      0.86       251
           8       0.92      0.90      0.91       268
           9       0.98      0.89      0.93       248
          10       0.97      0.95      0.96       266

    accuracy                           0.85      2750
   macro avg       0.85      0.85      0.85      2750
weighted avg       0.85      0.85      0.85      2750



'\n    low accuracy of class 1 => social news\n'

In [24]:
"""
    columns => predicted class
    rows => original class
"""
print(label_encoder.classes_)
print("\n", "#" * 100, '\n')
print(confusion_matrix(y_test, y_pred))


['آموزشي' 'اجتماعي' 'اقتصادي' 'بهداشتي' 'تاريخي' 'سياسي' 'علمي' 'فرهنگي'
 'فقه و حقوق' 'مذهبي' 'ورزشي']

 #################################################################################################### 

[[212  11   0   1   0   2  12   3   0   0   0]
 [ 13 183  21   5   3   4   9   9   7   2   3]
 [  1  22 202   1   0   3   6   5   1   0   0]
 [  0  11   3 228   0   2   9   1   0   0   1]
 [  0   6   0   0 215  11   1   5   0   0   0]
 [  0  25  14   3   7 191   2   5  12   1   0]
 [ 10   9   3  26   1   2 162   8   1   0   1]
 [  3  11   1   0   1   3   3 226   0   1   2]
 [  0  13   4   0   2   6   0   3 240   0   0]
 [  0  15   3   0   1   3   0   5   0 221   0]
 [  1   5   0   3   0   1   1   2   0   0 253]]
